In [1]:
import numpy as np
import sys
from pathlib import Path

PROJECT_ROOT = Path.cwd().resolve().parent
sys.path.insert(0, str(PROJECT_ROOT))

from mcda.matrices import PerformanceTable
from mcda.scales import QuantitativeScale, PreferenceDirection
from mcda.outranking.electre import Electre3

from methods.crisp_complete import crisp_complete_pulp
from methods.crisp_partial import crisp_partial_pulp

In [2]:
labels = ["A1", "A2", "A3", "A4", "A5", "A6", "A7"]

weights = [8, 4, 7, 2, 5]

# Definicja alternatyw
alternatives = [
    "A1",
    "A2",
    "A3",
    "A4",
    "A5",
    "A6",
    "A7"
]

# Definicja skali dla każdego kryterium
scales = {
    0: QuantitativeScale(1000, 4000, preference_direction=PreferenceDirection.MIN),# Price 
    1: QuantitativeScale(32, 1024, preference_direction=PreferenceDirection.MAX),    # Memory Size
    2: QuantitativeScale(8, 120, preference_direction=PreferenceDirection.MAX),     # Camera 
    3: QuantitativeScale(0, 5, preference_direction=PreferenceDirection.MAX),      # Functionalities 
    4: QuantitativeScale(12, 32, preference_direction=PreferenceDirection.MAX),      # Battery life
}

cutoff = 0.7

dataset = PerformanceTable(
    [[2200, 256, 32, 2, 24], [3200, 128, 64, 2, 18], [4000, 128, 32, 3, 22], [2400, 64, 32, 2, 28], [1400, 256, 120, 4, 14], [1600, 128, 120, 3, 14], [3200, 512, 16, 2, 20]],
    alternatives=alternatives, scales=scales
)
# Wagi kryteriów
W = {
    0: weights[0],  # Price 
    1: weights[1],   # Memory Size
    2: weights[2],  # Camera 
    3: weights[3],  # Functionalities 
    4: weights[4]   # Battery life
}

# Parametry Electre III

I = {0: 300, 1: 64,  2: 12, 3: 2,      4: 2}
P = {0: 600, 1: 256, 2: 32, 3: 3,      4: 6}
V = {0: 900, 1: 512, 2: 64, 3: np.inf, 4: 12}

In [3]:
electre3 = Electre3(performance_table=dataset, 
                    criteria_weights=W, 
                    indifference_thresholds=I, 
                    preference_thresholds=P, 
                    veto_thresholds=V)

In [4]:
credibility = electre3.credibility().data.values
crisp_matrix = (credibility >= cutoff).astype(int)

In [12]:
print(crisp_matrix)

[[1 1 1 1 0 0 1]
 [0 1 1 0 0 0 1]
 [0 0 1 0 0 0 0]
 [1 1 1 1 0 0 1]
 [1 1 1 0 1 1 0]
 [1 1 1 0 1 1 0]
 [0 1 1 0 0 0 1]]


In [ ]:
r_cc, z_cc, obj_cc = crisp_complete_pulp(crisp_matrix)

print("CRISP COMPLETE\n")
print(r_cc)
print(obj_cc)

CRISP COMPLETE

[[0 1 1 0 0 0 1]
 [0 0 1 0 0 0 1]
 [0 0 0 0 0 0 0]
 [1 1 1 0 1 1 1]
 [1 1 1 1 0 1 1]
 [1 1 1 1 1 0 1]
 [0 1 1 0 0 0 0]]
12.0


In [11]:
r_cp, Pp_cp, Pm_cp, I_cp, R_cp, obj_cp = crisp_partial_pulp(crisp_matrix)

print("CRISP PARTIAL\n")
print(r_cc)
print(obj_cp)

CRISP PARTIAL

[[0 1 1 0 0 0 1]
 [0 0 1 0 0 0 1]
 [0 0 0 0 0 0 0]
 [1 1 1 0 1 1 1]
 [1 1 1 1 0 1 1]
 [1 1 1 1 1 0 1]
 [0 1 1 0 0 0 0]]
7.0
